### Libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde, boxcox
import unicodedata
import re, json

In [2]:
from ingest.catalog import DatasetCatalog
from ingest.loader import RawDatasetLoader
from ingest.fetch.csv import CsvAdapter
from ingest.fetch.sct import SocrataAdapter
from ingest.fetch.excel import ExcelAdapter
from ingest.fetch.metadata import SocrataMetadata

In [3]:
from utils.clean import clean_text, is_numeric_string, normalize_code_to_length

In [4]:
## servicios
catalog = DatasetCatalog()
loader = RawDatasetLoader(
    csv_adapter=CsvAdapter(),
    sct_adapter=SocrataAdapter(),
    excel_adapter=ExcelAdapter(),
)
from config import  DATA_PATH

### Info base: empresas_10k

In [5]:
ds = catalog.get("empresas_10k")
meta = loader.metadata(ds)
if isinstance(meta, SocrataMetadata):
    print("Nombre:", meta.name)
    print("Descripción:", meta.description)
    print("Columnas:")
    columns = meta.columns
    for col in columns:
        print(f"  - {col.name} ({col.fieldName}): {col.dataTypeName}")
else:
    print("Este origen no soporta metadata estructurada.")
records = list(loader.load(ds))

Nombre: 10.000 Empresas mas Grandes del País
Descripción: Reporte de las diez mil empresas más grandes del país, todas las cifras en este reporte se muestran en Billones de Pesos. Lo invitamos a participar en la siguiente encuesta https://forms.office.com/r/1cWttkdHqM
Columnas:
  - NIT (nit): number
  - RAZÓN SOCIAL (raz_n_social): text
  - SUPERVISOR (supervisor): text
  - REGIÓN (regi_n): text
  - DEPARTAMENTO DOMICILIO (departamento_domicilio): text
  - CIUDAD DOMICILIO (ciudad_domicilio): text
  - CIIU (ciiu): number
  - MACROSECTOR (macrosector): text
  - INGRESOS OPERACIONALES (ingresos_operacionales): text
  - GANANCIA (PÉRDIDA) (ganancia_p_rdida): text
  - TOTAL ACTIVOS (total_activos): text
  - TOTAL PASIVOS (total_pasivos): text
  - TOTAL PATRIMONIO (total_patrimonio): text
  - Año de Corte (a_o_de_corte): number


In [6]:
df_10k = pd.DataFrame(records)
df_10k.describe(include="all")

,nit,raz_n_social,supervisor,regi_n,departamento_domicilio,ciudad_domicilio,ciiu,macrosector,ingresos_operacionales,ganancia_p_rdida,total_activos,total_pasivos,total_patrimonio,a_o_de_corte
count,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000,40000
unique,13976,18617,6,7,41,657,418,6,454,211,576,399,491,4
top,899999068,CONVIAS SAS,SUPERSOCIEDADES,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,8610,COMERCIO,$0.02,$0.00,$0.01,$0.01,$0.01,2022
freq,4,8,34834,18637,15749,11310,1374,13617,8311,25135,7669,12250,12258,10000


In [7]:
df_10k.head()

,nit,raz_n_social,supervisor,regi_n,departamento_domicilio,ciudad_domicilio,ciiu,macrosector,ingresos_operacionales,ganancia_p_rdida,total_activos,total_pasivos,total_patrimonio,a_o_de_corte
0,899999068,ECOPETROL S.A,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,610,MINERO,$144.82,$33.41,$216.85,$125.81,$91.03,2022
1,900112515,REFINERIA DE CARTAGENA S.A.S,SUPERSOCIEDADES,Costa Atlántica,BOLIVAR,CARTAGENA-BOLIVAR,1921,MANUFACTURA,$27.86,$2.19,$42.84,$16.48,$26.36,2022
2,830095213,ORGANIZACIÓN TERPEL S.A.,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,4661,COMERCIO,$23.60,$0.33,$7.48,$4.47,$3.01,2022
3,860069804,CARBONES DEL CERREJON LIMITED,SUPERSOCIEDADES,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,510,MINERO,$16.39,$6.05,$10.45,$9.00,$1.45,2022
4,800021308,DRUMMOND LTD,SUPERSOCIEDADES,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,510,MINERO,$15.27,$2.16,$14.27,$6.34,$7.93,2022


In [8]:
df_10k.groupby("a_o_de_corte")["nit"].nunique()

a_o_de_corte
2021    10000
2022    10000
2023    10000
2024    10000
Name: nit, dtype: int64

### Nit y Razon Social

**Reglas**
- Relación empresa - razon social, debe ser 1 a 1
- Son las primeras 10k empresas por cada año, no implica que sean las mismas

**Notes**
- Parecen haber empresas con diferentes NIT pero misma razón social o viceversa


In [9]:
def choose_base_from_names(names: list) -> str:
    """
    - Si hay strings, el base será el string más frecuente.
    - Si todos son numéricos, base = número más frecuente.
    """
    values = list(names)
    numeric = [n for n in values if is_numeric_string(n)]
    stringy = [n for n in values if not is_numeric_string(n)]

    if stringy:
        counts = pd.Series(stringy).value_counts()
        return counts.idxmax()
    counts = pd.Series(numeric).value_counts()
    return counts.idxmax()

def detect_identity_inconsistencies(df, id_col, name_col):
    # ID → nombres
    id_groups = (
        df.groupby(id_col)
        .agg(
            names=(name_col, lambda x: sorted(set(x))),
            n_names=(name_col, "nunique")
        )
        .reset_index()
    )
    inconsistent_id = id_groups[id_groups["n_names"] > 1]
    # Nombre → IDs
    name_groups = (
        df.groupby(name_col)
        .agg(
            ids=(id_col, lambda x: sorted(set(x))),
            n_ids=(id_col, "nunique")
        )
        .reset_index()
    )
    inconsistent_name = name_groups[name_groups["n_ids"] > 1]
    return inconsistent_id, inconsistent_name


In [10]:
def choose_base_dynamic(group, name_col, id_col, year_col):
    if year_col in group.columns:
        max_year = group[year_col].max()
        sub = group[group[year_col] == max_year]
        
        cleaned = [clean_text(n) for n in sub[name_col] if clean_text(n)]
        if cleaned:
            return pd.Series(cleaned).value_counts().idxmax()    
    cleaned_all = [clean_text(n) for n in group[name_col].unique() if clean_text(n)]
    if cleaned_all:
        return pd.Series(cleaned_all).value_counts().idxmax()
    return group[id_col].iloc[0]

def build_base_names(df, id_col, name_col, year_col="a_o_de_corte"):
    base_names = {}
    for nit, group in df.groupby(id_col):
        base = choose_base_dynamic(group, name_col, id_col, year_col)
        base_names[nit] = base
    return base_names


def apply_stage1(df, id_col, base_names):
    df["name_stage1"] = df[id_col].map(base_names)
    return df

def detect_multi_nit_names(df, name_col, id_col):
    name_groups = (
        df.groupby(name_col)
        .agg(nits=(id_col, lambda x: sorted(set(x))), n_ids=(id_col, "nunique"))
        .reset_index()
    )
    multi = name_groups[name_groups["n_ids"] > 1]
    return multi

def apply_stage2(df, id_col, name_col, multi):
    multi_names = set(multi[name_col].tolist())
    
    df["canonical_name"] = df.apply(
        lambda row:
            f"{row[name_col]}__{row[id_col]}"
            if row[name_col] in multi_names
            else row[name_col],
        axis=1
    )
    return df

In [11]:
## Nit y Razon Social
# convertir a string
df_10k.nit = df_10k.nit.astype(str).str.strip()
dups = df_10k.duplicated(subset=["nit", "a_o_de_corte"])
print(f"Nit duplicados por año: {dups.sum()}")
# verificar duplicados por año

# normalizar razon social
df_10k.raz_n_social = df_10k.apply(
    lambda row: clean_text(row.raz_n_social) or row.nit, axis=1
)
dups = df_10k.duplicated(subset=["raz_n_social", "a_o_de_corte"])
print(f"Razon social duplicados por año: {dups.sum()}")

inc_id, inc_name = detect_identity_inconsistencies(
    df_10k, id_col="nit", name_col="raz_n_social"
)
print(f"IDs con nombres múltiples: {len(inc_id)}")
print(f"Nombres con IDs múltiples: {len(inc_name)}")

base_names = build_base_names(df_10k, "nit", "raz_n_social")
df_10k = apply_stage1(df_10k, "nit", base_names)
multi = detect_multi_nit_names(df_10k, "name_stage1", "nit")
df_10k = apply_stage2(df_10k, "nit", "name_stage1", multi)
df_10k.drop(columns=["name_stage1", "raz_n_social"], inplace=True)
df_10k.rename(columns={"canonical_name": "raz_n_social"}, inplace=True)

inc_id, inc_name = detect_identity_inconsistencies(
    df_10k, id_col="nit", name_col="raz_n_social"
)

dups = df_10k.duplicated(subset=["raz_n_social", "a_o_de_corte"])
print(f"Razon social duplicados por año: {dups.sum()}")

print(f"IDs con nombres múltiples (After): {len(inc_id)}")
print(f"Nombres con IDs múltiples (After): {len(inc_name)}")
df_10k[["nit", "raz_n_social"]].describe()

Nit duplicados por año: 0
Razon social duplicados por año: 10
IDs con nombres múltiples: 3382
Nombres con IDs múltiples: 9
Razon social duplicados por año: 0
IDs con nombres múltiples (After): 0
Nombres con IDs múltiples (After): 0


,nit,raz_n_social
count,40000,40000
unique,13976,13976
top,899999068,ECOPETROL S.A
freq,4,4


### Financieras

In [12]:
financial_cols = [
    "ingresos_operacionales",
    "ganancia_p_rdida",
    "total_activos",
    "total_pasivos",
    "total_patrimonio"
]

for c in financial_cols:
    df_10k[c] = df_10k[c].astype(str)\
                 .str.replace("$", "", regex=False)\
                 .str.replace(".", "", regex=False)\
                 .str.replace(",", ".", regex=False)\
                 .astype(float) 
    df_10k[c] = pd.to_numeric(df_10k[c], errors='coerce')
df_10k[financial_cols].describe()

,ingresos_operacionales,ganancia_p_rdida,total_activos,total_pasivos,total_patrimonio
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,16.766950,1.316000,21.806125,11.070175,10.707975
std,139.299075,25.404198,236.961751,134.516182,111.752784
min,1.000000,-321.000000,0.000000,0.000000,-369.000000
25%,3.000000,0.000000,2.000000,1.000000,1.000000
50%,4.000000,0.000000,3.000000,2.000000,1.000000
75%,10.000000,1.000000,9.000000,5.000000,4.000000
max,14482.000000,3341.000000,21685.000000,13054.000000,9103.000000


### CIIU y Macrosector

In [29]:
# normalizar ciiu
df_10k["ciiu_code"] = df_10k["ciiu"].apply(lambda x: normalize_code_to_length(x, 4))
ds = catalog.get("ciiu")
records = list(loader.load(ds))
df_ciiu = pd.DataFrame(records)

In [30]:
ciiu_valid_set = set(
    df_ciiu["ciiu_code"]
        .dropna()
        .astype(str)
        .unique()
)

ciiu_10k_set = set(
    df_10k["ciiu_code"]
        .dropna()
        .astype(str)
        .unique()
)

ciiu_valid    = ciiu_10k_set.intersection(ciiu_valid_set)
ciiu_missing  = ciiu_10k_set.difference(ciiu_valid_set)

print("Total distinct in df_10k:", len(ciiu_10k_set))
print("Valid in catalog:", len(ciiu_valid))
print("Missing:", len(ciiu_missing))
print("Missing codes:", ciiu_missing)

def impute_missing_ciiu(ciiu_code: str, valid_codes: set) -> str:
    if ciiu_code in valid_codes:
        return ciiu_code

    code = str(ciiu_code).strip()
    if not code.isdigit():
        return None
    
    group_prefix = code[:3]  # 1031 -> "103"
    group_matches = sorted([c for c in valid_codes if c.startswith(group_prefix)])
    if group_matches: return group_matches[0]  # 1030
    return None  

df_10k["ciiu_code"] = df_10k["ciiu_code"].apply(
    lambda x: impute_missing_ciiu(x, ciiu_valid_set)
)

ciiu_10k_after = set(df_10k["ciiu_code"].dropna().astype(str).unique())
ciiu_new_missing = ciiu_10k_after.difference(ciiu_valid_set)
print("\nPost-imputation analysis:")
print("Total distinct now:", len(ciiu_10k_after))
print("Missing after imput:", len(ciiu_new_missing))
print("Missing codes after imput:", ciiu_new_missing)


Total distinct in df_10k: 418
Valid in catalog: 412
Missing: 6
Missing codes: {'1031', '7112', '7111', '1033', '1032', '8899'}

Post-imputation analysis:
Total distinct now: 412
Missing after imput: 0
Missing codes after imput: set()


In [31]:
map_ciiu_to_macro = (
    df_ciiu
        .dropna(subset=["ciiu_code"])
        .set_index("ciiu_code")["infered_macrosector"]
        .to_dict()
)
df_10k["macrosector_official"] = df_10k["ciiu_code"].map(map_ciiu_to_macro)
df_10k["macro_consistency"] = (
    df_10k["macrosector"] == df_10k["macrosector_official"]
)
df_incons = df_10k[df_10k["macro_consistency"] == False]
print(f"Registros con inconsistencia macrosector: {len(df_incons)}")
print(df_incons[["ciiu_code", "macrosector", "macrosector_official"]].head())

Registros con inconsistencia macrosector: 129
      ciiu_code   macrosector macrosector_official
275        6613   MANUFACTURA            SERVICIOS
320        6613   MANUFACTURA            SERVICIOS
459        5612   MANUFACTURA            SERVICIOS
1172       7110  CONSTRUCCIÓN            SERVICIOS
10076      4631   MANUFACTURA             COMERCIO


In [32]:
df_10k

,nit,supervisor,regi_n,departamento_domicilio,ciudad_domicilio,ciiu,macrosector,ingresos_operacionales,ganancia_p_rdida,total_activos,total_pasivos,total_patrimonio,a_o_de_corte,raz_n_social,ciiu_code,macrosector_official,macro_consistency,regi_n_corregida,region_oficial
0,899999068,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,610,MINERO,14482.0,3341.0,21685.0,12581.0,9103.0,2022,ECOPETROL S.A,0610,MINERO,True,Bogotá - Cundinamarca,Andina
1,900112515,SUPERSOCIEDADES,Costa Atlántica,BOLIVAR,CARTAGENA-BOLIVAR,1921,MANUFACTURA,2786.0,219.0,4284.0,1648.0,2636.0,2022,REFINERIA DE CARTAGENA S.A.,1921,MANUFACTURA,True,Costa Atlántica,Caribe
2,830095213,SUPERFINANCIERA,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,4661,COMERCIO,2360.0,33.0,748.0,447.0,301.0,2022,ORGANIZACION TERPEL S.A.,4661,COMERCIO,True,Bogotá - Cundinamarca,Andina
3,860069804,SUPERSOCIEDADES,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,510,MINERO,1639.0,605.0,1045.0,900.0,145.0,2022,CARBONES DEL CERREJON LIMITED,0510,MINERO,True,Bogotá - Cundinamarca,Andina
4,800021308,SUPERSOCIEDADES,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,510,MINERO,1527.0,216.0,1427.0,634.0,793.0,2022,DRUMMOND LTD,0510,MINERO,True,Bogotá - Cundinamarca,Andina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,901266264,SUPERSOCIEDADES,Costa Atlántica,BOLIVAR,CARTAGENA-BOLIVAR,4719,COMERCIO,2.0,0.0,0.0,0.0,0.0,2024,ALIA2 DISTRIBUCIONES S.A.S.,4719,COMERCIO,True,Costa Atlántica,Caribe
39996,900584429,SUPERSOCIEDADES,Costa Atlántica,ATLANTICO,BARRANQUILLA-ATLANTICO,1709,MANUFACTURA,2.0,0.0,1.0,0.0,1.0,2024,C I UNIPACK COLOMBIA SAS,1709,MANUFACTURA,True,Costa Atlántica,Caribe
39997,807009474,SUPERSOCIEDADES,Centro - Oriente,NORTE DE SANTANDER,CUCUTA-NORTE DE SANTANDER,4645,COMERCIO,2.0,0.0,1.0,0.0,0.0,2024,DROGAS UNIDAS SAS,4645,COMERCIO,True,Centro - Oriente,Andina
39998,830066984,SUPERSOCIEDADES,Bogotá - Cundinamarca,BOGOTA D.C.,BOGOTA D.C.-BOGOTA D.C.,4390,CONSTRUCCIÓN,2.0,-0.0,4.0,6.0,-2.0,2024,REDES Y PROYECTOS DE ENERGIA S A EMPRESA MULTI...,4390,CONSTRUCCIÓN,True,Bogotá - Cundinamarca,Andina


In [28]:
# Número de CIIU distintos por empresa
ciiu_change = (
    df_10k.groupby("nit")["ciiu"]
    .nunique()
    .reset_index(name="ciiu_variants")
    .query("ciiu_variants > 1")
)

print(f"Empresas con múltiples CIIU: {len(ciiu_change)}")

ciiu_details = (
    df_10k[df_10k["nit"].isin(ciiu_change["nit"])]
    .sort_values(["nit", "a_o_de_corte"])
    [["nit", "a_o_de_corte", "ciiu", "macrosector"]]
)

sector_change = (
    df_10k.groupby("nit")["macrosector"]
    .nunique()
    .reset_index(name="sector_variants")
    .query("sector_variants > 1")
)

print(f"Empresas con cambios de macrosector: {len(sector_change)}")

sector_details = (
    df_10k[df_10k["nit"].isin(sector_change["nit"])]
    .sort_values(["nit", "a_o_de_corte"])
    [["nit", "a_o_de_corte", "macrosector"]]
)

Empresas con múltiples CIIU: 694
Empresas con cambios de macrosector: 294


### Region: Departamento y Ciudad

In [18]:
ds = catalog.get("divipola")
meta = loader.metadata(ds)
records = list(loader.load(ds))
df_div = pd.DataFrame(records)
df_div.head(3)

,cod_dpto,dpto,cod_mpio,nom_mpio,tipo_municipio,longitud,latitud
0,05,ANTIOQUIA,05001,MEDELLÍN,Municipio,"-75,581775","6,246631"
1,05,ANTIOQUIA,05002,ABEJORRAL,Municipio,"-75,428739","5,789315"
2,05,ANTIOQUIA,05004,ABRIAQUÍ,Municipio,"-76,064304","6,632282"


In [19]:
df_10k.departamento_domicilio = df_10k.apply(
    lambda row: clean_text(row.departamento_domicilio), axis=1
)
unique_dept = df_10k.departamento_domicilio.unique()
print(f"Departamentos unicos: {len(unique_dept)}")

DEPT_CORRECTIONS = {
    "VALLE": "VALLE DEL CAUCA",     
    "GUAJIRA": "LA GUAJIRA",
    "MONTERIA": "CORDOBA",
    "VALLEDUPAR": "CESAR",
    "ARMENIA": "QUINDIO",
}

def normalize_department(s):
    if pd.isna(s):
        return None
    s_clean = clean_text(s)
    return DEPT_CORRECTIONS.get(s_clean, s_clean)

df_10k["departamento_domicilio"] = df_10k["departamento_domicilio"].apply(normalize_department)
df_10k.departamento_domicilio.value_counts()
unique_dept = df_10k.departamento_domicilio.unique()
print(f"Departamentos unicos: {len(unique_dept)} (After)")

Departamentos unicos: 38
Departamentos unicos: 33 (After)


#### Las regiones originales vs departamentos parecen tener casos imposibles o extraños
- Region con departamentos que no le corresponden

In [20]:
reg_dep = {
    region: sorted(
        df_10k.loc[df_10k["regi_n"] == region, "departamento_domicilio"].unique().tolist()
    )
    for region in df_10k.regi_n.unique()
}
print(json.dumps(reg_dep, indent=4, ensure_ascii=False))

{
    "Bogotá - Cundinamarca": [
        "BOGOTA D.C.",
        "CASANARE",
        "CUNDINAMARCA",
        "META",
        "NORTE DE SANTANDER",
        "SANTANDER"
    ],
    "Costa Atlántica": [
        "ATLANTICO",
        "BOGOTA D.C.",
        "BOLIVAR",
        "CESAR",
        "CORDOBA",
        "CUNDINAMARCA",
        "LA GUAJIRA",
        "MAGDALENA",
        "SAN ANDRES Y PROVIDENCIA",
        "SUCRE"
    ],
    "Antioquia": [
        "ANTIOQUIA",
        "ATLANTICO",
        "BOGOTA D.C.",
        "CALDAS",
        "VALLE DEL CAUCA"
    ],
    "Costa Pacífica": [
        "BOGOTA D.C.",
        "CAUCA",
        "CHOCO",
        "NARINO",
        "VALLE DEL CAUCA"
    ],
    "Eje Cafetero": [
        "ANTIOQUIA",
        "BOGOTA D.C.",
        "CALDAS",
        "QUINDIO",
        "RISARALDA"
    ],
    "Centro - Oriente": [
        "ANTIOQUIA",
        "BOYACA",
        "NORTE DE SANTANDER",
        "SANTANDER"
    ],
    "Otros": [
        "AMAZONAS",
        "ARAUCA",
     

In [21]:
# mapa correcto departamento -> región (segun su propios tipos)
correct = {
    # Bogotá - Cundinamarca
    "BOGOTA D.C.": "Bogotá - Cundinamarca",
    "CUNDINAMARCA": "Bogotá - Cundinamarca",

    # Centro - Oriente
    "BOYACA": "Centro - Oriente",
    "TOLIMA": "Centro - Oriente",
    "HUILA": "Centro - Oriente",
    "SANTANDER": "Centro - Oriente",
    "NORTE DE SANTANDER": "Centro - Oriente",

    # Costa Atlántica
    "ATLANTICO": "Costa Atlántica",
    "MAGDALENA": "Costa Atlántica",
    "BOLIVAR": "Costa Atlántica",
    "SUCRE": "Costa Atlántica",
    "CORDOBA": "Costa Atlántica",
    "LA GUAJIRA": "Costa Atlántica",
    "CESAR": "Costa Atlántica",

    # Costa Pacífica
    "VALLE DEL CAUCA": "Costa Pacífica",
    "CAUCA": "Costa Pacífica",
    "NARINO": "Costa Pacífica",
    "CHOCO": "Costa Pacífica",

    # ANTIOQUIA
    "ANTIOQUIA": "Antioquia",

    # Eje Cafetero
    "CALDAS": "Eje Cafetero",
    "RISARALDA": "Eje Cafetero",
    "QUINDIO": "Eje Cafetero",

    # Otros (Amazonía – Orinoquía)
    "SAN ANDRES Y PROVIDENCIA": "Otros",
    "META": "Otros",
    "CASANARE": "Otros",
    "ARAUCA": "Otros",
    "GUAVIARE": "Otros",
    "GUAINIA": "Otros",
    "VAUPES": "Otros",
    "VICHADA": "Otros",
    "AMAZONAS": "Otros",
    "CAQUETA": "Otros",
    "PUTUMAYO": "Otros",
}
df_10k["regi_n_corregida"] = df_10k["departamento_domicilio"].map(correct)

In [22]:
region_oficial = {
    "Andina": [
        "BOGOTA D.C.",
        "ANTIOQUIA",
        "BOYACA",
        "CALDAS",
        "CUNDINAMARCA",
        "HUILA",
        "NORTE DE SANTANDER",
        "QUINDIO",
        "RISARALDA",
        "SANTANDER",
        "TOLIMA",
    ],

    "Caribe": [
        "ATLANTICO",
        "BOLIVAR",
        "CESAR",
        "CORDOBA",
        "LA GUAJIRA",
        "MAGDALENA",
        "SUCRE",
    ],

    "Pacifico": [
        "CAUCA",
        "CHOCO",
        "NARINO",
        "VALLE DEL CAUCA",
    ],

    "Orinoquia": [
        "ARAUCA",
        "CASANARE",
        "META",
        "VICHADA",
    ],

    "Amazonia": [
        "AMAZONAS",
        "CAQUETA",
        "GUAINIA",
        "GUAVIARE",
        "PUTUMAYO",
        "VAUPES",
    ],

    "Insular": [
        "SAN ANDRES Y PROVIDENCIA"
    ],
}

dept_to_region = {
    dept: region
    for region, dept_list in region_oficial.items()
    for dept in dept_list
}

df_10k["region_oficial"] = df_10k["departamento_domicilio"].map(dept_to_region)

- Elegir una region y cruzar con divipola -> asignar lat lon a cada departamento

In [23]:
inconsistent = df_10k[df_10k["regi_n"] != df_10k["regi_n_corregida"]]
inconsistent[["departamento_domicilio", "regi_n", "regi_n_corregida", "region_oficial"]].value_counts()

departamento_domicilio    regi_n                 regi_n_corregida       region_oficial
TOLIMA                    Otros                  Centro - Oriente       Andina            343
HUILA                     Otros                  Centro - Oriente       Andina            319
SAN ANDRES Y PROVIDENCIA  Costa Atlántica        Otros                  Insular            64
CESAR                     Otros                  Costa Atlántica        Caribe             60
CHOCO                     Otros                  Costa Pacífica         Pacifico           15
LA GUAJIRA                Otros                  Costa Atlántica        Caribe             15
BOGOTA D.C.               Costa Pacífica         Bogotá - Cundinamarca  Andina              6
                          Otros                  Bogotá - Cundinamarca  Andina              4
                          Antioquia              Bogotá - Cundinamarca  Andina              3
                          Costa Atlántica        Bogotá - Cundinama

In [24]:
region_change = (
    df_10k.groupby("nit")["regi_n_corregida"]
    .nunique()
    .reset_index(name="region_variants")
    .query("region_variants > 1")
)

print(f"Empresas con cambios de región: {len(region_change)}")

dpto_change = (
    df_10k.groupby("nit")["departamento_domicilio"]
    .nunique()
    .reset_index(name="dpto_variants")
    .query("dpto_variants > 1")
)
print(f"Empresas con cambios de departamento: {len(dpto_change)}")

dpto_change = (
    df_10k.groupby("nit")["ciudad_domicilio"]
    .nunique()
    .reset_index(name="dpto_variants")
    .query("dpto_variants > 1")
)
print(f"Empresas con cambios de ciudad: {len(dpto_change)}")

Empresas con cambios de región: 68
Empresas con cambios de departamento: 159
Empresas con cambios de ciudad: 6293
